# Fine-Tuning mBART on the United Nations Parallel Corpus (UNPC)

In this project we take the multilingual model mBart and fine-tune it for official document translation using an English-Chinese corpus of UN documents.


Pre-requisites

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [ ]:
pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install wandb

In [ ]:
import transformers
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
import json
import re
import torch
from tqdm import tqdm
import evaluate
import wandb
from huggingface_hub import notebook_login

# mBart

mBart is a multilingual encoder-decoder (sequence-to-sequence) model primarily intended for translation. It has the same architecture as BART, i.e., 12 encoder layers, 12 decoder layers, and a dimension of 1024.

In [ ]:
device="cuda"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
print(model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

# The United Nations Parallel Corpus v1.0

The UNPC corpus is built from parliamentary documents in the public domain of the United Nations from 1990-2014 and is built to facilitate the multilingual NLP tasks such as statistical machine translation (SMT). The English-Chinese subset of UNPC contains around 17.5 million sentence pairs. The dataset we use is from the HelsinkiNLP group. We are going to use it to fine-tune mBart to perform the task of official document translation.

References:

Michał Ziemski, Marcin Junczys-Dowmunt, and Bruno Pouliquen. 2016. The United Nations Parallel Corpus v1.0. In *Proceedings of the Tenth International Conference on Language Resources and Evaluation (LREC'16)*, pages 3530–3534, Portorož, Slovenia. European Language Resources Association (ELRA).

https://huggingface.co/datasets/Helsinki-NLP/un_pc


Importing the UNPC (en-zh) Dataset

In [ ]:
dataset = load_dataset("Helsinki-NLP/un_pc", "en-zh")
dataset

README.md:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

train-00000-of-00010.parquet:   0%|          | 0.00/277M [00:00<?, ?B/s]

train-00001-of-00010.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

train-00002-of-00010.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

...

train-00007-of-00010.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

train-00008-of-00010.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00009-of-00010.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17451549 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 17451549
    })
})

In [ ]:
dataframe = pd.DataFrame(dataset['train'][0:100000]['translation'])
dataframe

,en,zh
0,RESOLUTION 918 (1994),第918(1994)号决议
1,Adopted by the Security Council at its 3377th ...,1994年5月17日安全理事会第3377次会议通过
2,"The Security Council,",安全理事会，
3,Reaffirming all its previous resolutions on th...,重申其以往关于卢旺达局势的所有决议，特别是成立联合国卢旺达援助团(联卢援助团)的1993年1...
4,Recalling the statements made by the President...,回顾安理会主席以安理会名义在1994年4月7日发表的声明(S/PRST/ 1994/16)和...
...,...,...
99995,(d) North American Symposium of Non-Government...,(d) 非政府组织北美洲座谈会
99996,87. The North American Symposium of Non-Govern...,87. 1997年6月9日至16日在联合国总部举行主题为 &quot; 三十年的占领:展望自...
99997,88. Its panel discussions dealt with the key i...,88. 小组讨论会审议了公正和全面解决的关键问题;国际社会在长期地位的过渡时期所发挥的作用以...
99998,Six workshops were held on related themes to m...,"就有关的主题举办了6个实习班,以便动员北美洲非政府组织采取具体行动。"


In [ ]:
ds = Dataset.from_pandas(dataframe)
ds

Dataset({
    features: ['en', 'zh'],
    num_rows: 100000
})

The dataset is splited into the training, validation, and testing dataset, with a proportion of 18:1:1.

In [ ]:
train_test = ds.train_test_split(test_size = 5000 / dataframe.shape[0],seed = 99)

In [ ]:
dataset_test = train_test["test"]

In [ ]:
target_encodings = tokenizer(train_test["train"]["zh"], max_length = 1024, truncation = True)
input_encodings = tokenizer(train_test["train"]["en"], max_length = 1024, truncation = True)
dataset_pt = Dataset.from_dict(
    {"input_ids": input_encodings["input_ids"],
     "attention_mask": input_encodings["attention_mask"],
     "labels": target_encodings["input_ids"]})
columns = ["input_ids", "labels", "attention_mask"]
dataset_pt.set_format(type = "torch", columns = columns)

In [ ]:
train_valid = dataset_pt.train_test_split(test_size = 5000 / (dataframe.shape[0] - 5000), seed = 99)

In [ ]:
dataset_train, dataset_valid = train_valid["train"], train_valid["test"]

In [ ]:
dataset_train, dataset_valid, dataset_test

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 90000
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 5000
 }),
 Dataset({
     features: ['en', 'zh'],
     num_rows: 5000
 }))

First of all, we will evaluate the performance of the vanilla mBart on a testing dataset of 5,000 sentence pairs using the BLEU metrics.

In [ ]:
torch.cuda.empty_cache()
model.to("cuda")
tokenizer.src_lang = "en_XX"
target_lang = "zh_CN"
forced_bos_token_id = tokenizer.lang_code_to_id[target_lang]
predictions_vanilla = []
batch_size = 16

for i in tqdm(range(0, len(dataset_test["en"]), batch_size)):
    batch_texts = dataset_test["en"][i:i+batch_size]

    inputs = tokenizer(batch_texts,
                       return_tensors = "pt",
                       padding = "longest",
                       truncation = True,
                       max_length = 256)

    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")

    outputs = model.generate(
        input_ids = input_ids,
        attention_mask = attention_mask,
        max_length = 256,
        num_beams =6,
        no_repeat_ngram_size = 2,
        repetition_penalty = 1.2,
        early_stopping = True,
        forced_bos_token_id = forced_bos_token_id
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Post-processing method to remove the spaces
    cleaned_predictions = [re.sub(r'\s+', '', pred.strip()) for pred in decoded]

    predictions_vanilla.extend(cleaned_predictions)

100%|██████████| 313/313 [34:07<00:00,  6.54s/it]


In [ ]:
# save the data
with open('predictions_vanilla.json', 'w', encoding = 'utf-8') as out:
    json.dump(predictions_vanilla, out, indent = 2, ensure_ascii = False)

In [ ]:
predictions_vanilla
references = [dataset_test["zh"]]
bleu = evaluate.load("bleu")
bleu.add(predictions = str(predictions_vanilla), references = str(references))
bleu.compute()

{'bleu': 0.2012569591711336,
 'precisions': [0.5385992902028457,
  0.24114352696014588,
  0.15541662596463807,
  0.08127645718007163],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0400609549610567,
 'translation_length': 30713,
 'reference_length': 29530}

Next we will fine-tune mBart using the training dataset.

In [ ]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

training_args = TrainingArguments(
    output_dir = "en-zh",
    num_train_epochs = 1,
    warmup_steps = 500,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    weight_decay = 0.01,
    logging_steps = 10,
    push_to_hub = False,
    eval_strategy = "steps",
    eval_steps = 30,
    save_steps = 600,
    gradient_accumulation_steps = 128,
    load_best_model_at_end = True,
)

trainer = Trainer(model = model,
                  args = training_args,
                  tokenizer = tokenizer,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_train,
                  eval_dataset = dataset_valid)

<ipython-input-28-bd0f86ad4d69>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model,


In [ ]:
notebook_login()
wandb.init(mode = "disabled")

In [ ]:
torch.cuda.empty_cache()
trainer.train()
trainer.save_model("mbart-unpc-en-zh")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
30,3.052400,2.691103
60,1.847400,1.660403
90,1.488700,1.476536
120,1.415800,1.369277
150,1.336000,1.315818
180,1.320800,1.277010
210,1.303300,1.256181
240,1.296300,1.235462
270,1.250800,1.225197
300,1.263300,1.213217


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3463: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


In [ ]:
# load the fine-tuned model after the training is completed
device="cuda"
model = MBartForConditionalGeneration.from_pretrained("./mbart-unpc-en-zh").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("./mbart-unpc-en-zh")

In [ ]:
# or download and load the fine-tuned model mBART-UNPC-en-zh (skip the training process)
!gdown 1Kd25Z7D39xdsaoZOD4XKHDZFnzuuIIT3
!gunzip mbart-UNPC-en-zh.tar.gz
!tar xf mbart-UNPC-en-zh.tar

We evaluate the fine-tuned model using BLEU.

In [ ]:
torch.cuda.empty_cache()
model.to("cuda")
tokenizer.src_lang = "en_XX"
target_lang = "zh_CN"
forced_bos_token_id = tokenizer.lang_code_to_id[target_lang]
predictions_finetuned = []
batch_size = 16

for i in tqdm(range(0, len(dataset_test["en"]), batch_size)):
    batch_texts = dataset_test["en"][i:i+batch_size]

    inputs = tokenizer(batch_texts,
                       return_tensors = "pt",
                       padding = "longest",
                       truncation = True,
                       max_length = 256)

    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")

    outputs = model.generate(
        input_ids = input_ids,
        attention_mask = attention_mask,
        max_length = 256,
        num_beams =6,
        no_repeat_ngram_size = 2,
        repetition_penalty = 1.2,
        early_stopping = True,
        forced_bos_token_id = forced_bos_token_id
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Post-processing method to remove the spaces
    cleaned_predictions = [re.sub(r'\s+', '', pred.strip()) for pred in decoded]

    predictions_finetuned.extend(cleaned_predictions)

100%|██████████| 313/313 [18:01<00:00,  3.45s/it]


In [ ]:
# save the data
with open('predictions_finetuned.json', 'w', encoding = 'utf-8') as out:
    json.dump(predictions_finetuned, out, indent = 2, ensure_ascii = False)

In [ ]:
# check the generated translation and compare with the reference translation.
for i in range(100):
    print("Input:", dataset_test["en"][i])
    print("Generated:", predictions_finetuned[i])
    print("Vanilla:", predictions_vanilla[i])
    print("Reference:", dataset_test["zh"][i])
    print("=" * 80)

Input: Moreover, the trial judge was allegedly biased against the author and made ironical and provocative comments on the sentence, which were later criticized by the Court of Appeal and which, according to the author, provide further evidence of the court &apos; s bias.
Generated: 此外,审判法官据称偏向于提交人,对判决作出讽刺和挑衅性的评论,后来被上诉法院批评,根据提交人的说法,这进一步证实了法院的偏见。
Vanilla: 此外,审判法官据称对撰文人有偏见,对判决作了讽刺和挑衅性的评论,后来被上诉法院批评,并据撰述者说,这些评论进一步证明了法院的偏爱。
Reference: 此外,初审法官据称对提交人有偏见,并以挖苦和挑衅的口气谈论判决,这一点后来遭到上诉法院的批评,提交人认为,这一点再次证明法院有偏见。
Input: See Council resolution 1996/7 (para. 72 above).
Generated: 见理事会第1996/7号决议(上文第72段)。
Vanilla: 见理事会第1996/7号决议(上文第72段)。
Reference: 见理事会第1996/7 号决议(上文第72段)。
Input: Additional prohibitions:
Generated: 其他禁止:
Vanilla: 其他禁令:
Reference: 此外，还禁止：
Input: 366. During the period under review, no new cases of disappearance were transmitted by the Working Group to the Government of Uzbekistan.
Generated: 366.在审查期间,工作组没有向乌兹别克斯坦政府转交新的失踪案件。
Vanilla: 366.在本报告所述期间,工作组没有向乌兹别克斯坦政府转递新的失踪案件。
Reference: 366. 在审查期间

In [ ]:
import evaluate
predictions_finetuned
references = [dataset_test["zh"]]
bleu = evaluate.load("bleu")
bleu.add(predictions = str(predictions_finetuned), references=str(references))
bleu.compute()

{'bleu': 0.21528613234128932,
 'precisions': [0.5065294117647059,
  0.25130150886790786,
  0.17024530854756162,
  0.09912639350530929],
 'brevity_penalty': 1.0,
 'length_ratio': 1.1513714866237725,
 'translation_length': 34000,
 'reference_length': 29530}